In [22]:
# 시간 제한 있음

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

def distance(city1, city2):
    return np.linalg.norm(np.array(city1) - np.array(city2))

def pctsp_with_rewards(coordinates, rewards, max_distance):
    city_count = len(coordinates)
    data = {
        'city_count': city_count,
        'max_distance': max_distance,
        'city_locations': coordinates,
        'city_rewards': rewards
    }

    tsp_size = data['city_count']
    num_routes = 1
    depot = 0  # 시작 도시

    manager = pywrapcp.RoutingIndexManager(tsp_size, num_routes, depot)
    routing = pywrapcp.RoutingModel(manager)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    def distance_callback(from_index, to_index):
        from_city = manager.IndexToNode(from_index)
        to_city = manager.IndexToNode(to_index)
        return distance(data['city_locations'][from_city], data['city_locations'][to_city])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def reward_callback(from_index):
        return data['city_rewards'][manager.IndexToNode(from_index)]

    reward_callback_index = routing.RegisterUnaryTransitCallback(reward_callback)
    routing.AddDimensionWithVehicleCapacity(
        reward_callback_index,
        0,  # no slack
        [max_distance],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Reward'
    )

    search_parameters.time_limit.seconds = 10  # 시간 제한 설정 (원하는 값으로 변경 가능)

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        route_distance = 0
        route_reward = 0
        index = routing.Start(0)
        while not routing.IsEnd(index):
            city_idx = manager.IndexToNode(index)
            next_city_idx = manager.IndexToNode(solution.Value(routing.NextVar(index)))
            route_distance += distance(data['city_locations'][city_idx], data['city_locations'][next_city_idx])
            route_reward += data['city_rewards'][city_idx]
            index = solution.Value(routing.NextVar(index))

        return route_distance, route_reward
    else:
        return "No solution found"

coordinates =  [[0.8399, 0.3007],
        [0.4468, 0.0890],
        [0.4920, 0.6809],
        [0.6088, 0.2040],
        [0.6129, 0.1492],
        [0.4122, 0.7854],
        [0.5306, 0.3192],
        [0.1819, 0.5648],
        [0.7146, 0.9010],
        [0.0962, 0.0210],
        [0.6243, 0.8617],
        [0.1295, 0.2442],
        [0.9077, 0.2649],
        [0.5887, 0.7566],
        [0.6873, 0.3199],
        [0.4174, 0.9208],
        [0.4273, 0.9045],
        [0.3627, 0.8007],
        [0.2802, 0.5079],
        [0.1866, 0.2817],
        [0.0339, 0.0011],
        [0.6550, 0.1079],
        [0.3038, 0.8925],
        [0.3303, 0.3060],
        [0.3070, 0.1467],
        [0.3492, 0.1632],
        [0.4071, 0.1213],
        [0.3865, 0.6832],
        [0.7788, 0.6352],
        [0.7612, 0.2668],
        [0.5506, 0.9410],
        [0.8515, 0.6281],
        [0.2919, 0.9658],
        [0.3349, 0.3588],
        [0.6571, 0.7825],
        [0.6022, 0.0432],
        [0.0614, 0.3466],
        [0.8276, 0.9415],
        [0.7648, 0.6016],
        [0.3539, 0.4902],
        [0.6788, 0.4456],
        [0.6091, 0.6261],
        [0.0674, 0.0811],
        [0.5408, 0.2559],
        [0.8533, 0.0023],
        [0.7900, 0.4175],
        [0.5529, 0.2516],
        [0.4436, 0.2833],
        [0.0325, 0.0038],
        [0.1123, 0.6989],
        [0.1659, 0.2742],
        [0.2222, 0.0554],
        [0.7133, 0.1792],
        [0.3699, 0.7700],
        [0.0114, 0.7788],
        [0.2502, 0.1281],
        [0.5050, 0.8515],
        [0.8470, 0.0188],
        [0.5241, 0.4316],
        [0.2861, 0.7774],
        [0.0942, 0.6031],
        [0.5943, 0.2128],
        [0.2495, 0.3538],
        [0.6099, 0.0682],
        [0.0751, 0.1698],
        [0.3069, 0.8234],
        [0.6540, 0.3656],
        [0.6869, 0.3879],
        [0.5544, 0.1854],
        [0.1205, 0.7324],
        [0.8193, 0.6112],
        [0.2600, 0.0815],
        [0.0968, 0.8144],
        [0.2054, 0.6480],
        [0.4056, 0.3024],
        [0.9318, 0.1093],
        [0.7847, 0.8003],
        [0.3071, 0.7083],
        [0.7684, 0.2580],
        [0.3717, 0.7092],
        [0.9891, 0.7555],
        [0.2962, 0.7518],
        [0.3365, 0.1623],
        [0.6916, 0.9256],
        [0.1574, 0.4046],
        [0.7143, 0.3958],
        [0.3072, 0.6785],
        [0.3442, 0.8242],
        [0.4807, 0.5795],
        [0.7869, 0.2391],
        [0.8203, 0.9767],
        [0.6361, 0.5395],
        [0.9245, 0.5752],
        [0.0582, 0.3420],
        [0.4908, 0.6234],
        [0.8633, 0.9506],
        [0.9131, 0.1951],
        [0.0342, 0.4091],
        [0.6588, 0.2756],
        [0.1228, 0.9335]]  # 좌표값 리스트
rewards =  [40, 55, 60, 76, 42, 36,  9,  2, 51, 23, 33, 78, 27, 41, 84, 51,  6, 76, 38, 57, 71, 32, 98, 16, 99, 31, 46, 52, 56, 10,  2, 82, 37, 44, 23, 12,
        33, 86, 57, 58, 22, 85, 38, 34, 64, 42, 89, 50, 54, 77, 95, 12, 90, 11,52, 87, 47, 93, 71, 69,  3, 86, 21,  9, 86, 29, 65, 80, 11, 80, 61, 22,
        38, 93, 77, 91, 26,  7, 39,  2, 46, 16, 66, 54, 92,  4, 32, 84, 80, 73, 41, 56, 45, 84, 20, 52, 61, 30, 33,  3]   # 리워드 리스트
max_distance = 10

result = pctsp_with_rewards(coordinates, rewards, max_distance)
if isinstance(result, tuple):
    total_distance, total_reward = result
    print(f"Total Distance: {total_distance}")
    print(f"Total Reward: {total_reward}")
else:
    print(result)


No solution found


In [28]:
# 시간 제한 없음

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

def distance(city1, city2):
    return np.linalg.norm(np.array(city1) - np.array(city2))

def pctsp_with_rewards(coordinates, rewards, max_distance):
    city_count = len(coordinates)
    data = {
        'city_count': city_count,
        'max_distance': max_distance,
        'city_locations': coordinates,
        'city_rewards': rewards
    }

    tsp_size = data['city_count']
    num_routes = 1
    depot = 0  # 시작 도시

    manager = pywrapcp.RoutingIndexManager(tsp_size, num_routes, depot)
    routing = pywrapcp.RoutingModel(manager)
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()

    def distance_callback(from_index, to_index):
        from_city = manager.IndexToNode(from_index)
        to_city = manager.IndexToNode(to_index)
        return distance(data['city_locations'][from_city], data['city_locations'][to_city])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    def reward_callback(from_index):
        return data['city_rewards'][manager.IndexToNode(from_index)]

    reward_callback_index = routing.RegisterUnaryTransitCallback(reward_callback)
    routing.AddDimensionWithVehicleCapacity(
        reward_callback_index,
        0,  # no slack
        [max_distance],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Reward'
    )

    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        route_distance = 0
        route_reward = 0
        index = routing.Start(0)
        while not routing.IsEnd(index):
            city_idx = manager.IndexToNode(index)
            next_city_idx = manager.IndexToNode(solution.Value(routing.NextVar(index)))
            route_distance += distance(data['city_locations'][city_idx], data['city_locations'][next_city_idx])
            route_reward += data['city_rewards'][city_idx]
            index = solution.Value(routing.NextVar(index))

        return route_distance, route_reward
    else:
        return "No solution found"

coordinates =  [[0.8399, 0.3007],
                
        [0.4468, 0.0890],
        [0.4920, 0.6809],
        [0.6088, 0.2040],
        [0.6129, 0.1492],
        [0.4122, 0.7854],
        [0.5306, 0.3192],
        [0.1819, 0.5648],
        [0.7146, 0.9010],
        [0.0962, 0.0210],
        [0.6243, 0.8617],
        [0.1295, 0.2442],
        [0.9077, 0.2649],
        [0.5887, 0.7566],
        [0.6873, 0.3199],
        [0.4174, 0.9208],
        [0.4273, 0.9045],
        [0.3627, 0.8007],
        [0.2802, 0.5079],
        [0.1866, 0.2817],
        [0.0339, 0.0011],
        [0.6550, 0.1079],
        [0.3038, 0.8925],
        [0.3303, 0.3060],
        [0.3070, 0.1467],
        [0.3492, 0.1632],
        [0.4071, 0.1213],
        [0.3865, 0.6832],
        [0.7788, 0.6352],
        [0.7612, 0.2668],
        [0.5506, 0.9410],
        [0.8515, 0.6281],
        [0.2919, 0.9658],
        [0.3349, 0.3588],
        [0.6571, 0.7825],
        [0.6022, 0.0432],
        [0.0614, 0.3466],
        [0.8276, 0.9415],
        [0.7648, 0.6016],
        [0.3539, 0.4902],
        [0.6788, 0.4456],
        [0.6091, 0.6261],
        [0.0674, 0.0811],
        [0.5408, 0.2559],
        [0.8533, 0.0023],
        [0.7900, 0.4175],
        [0.5529, 0.2516],
        [0.4436, 0.2833],
        [0.0325, 0.0038],
        [0.1123, 0.6989],
        [0.1659, 0.2742],
        [0.2222, 0.0554],
        [0.7133, 0.1792],
        [0.3699, 0.7700],
        [0.0114, 0.7788],
        [0.2502, 0.1281],
        [0.5050, 0.8515],
        [0.8470, 0.0188],
        [0.5241, 0.4316],
        [0.2861, 0.7774],
        [0.0942, 0.6031],
        [0.5943, 0.2128],
        [0.2495, 0.3538],
        [0.6099, 0.0682],
        [0.0751, 0.1698],
        [0.3069, 0.8234],
        [0.6540, 0.3656],
        [0.6869, 0.3879],
        [0.5544, 0.1854],
        [0.1205, 0.7324],
        [0.8193, 0.6112],
        [0.2600, 0.0815],
        [0.0968, 0.8144],
        [0.2054, 0.6480],
        [0.4056, 0.3024],
        [0.9318, 0.1093],
        [0.7847, 0.8003],
        [0.3071, 0.7083],
        [0.7684, 0.2580],
        [0.3717, 0.7092],
        [0.9891, 0.7555],
        [0.2962, 0.7518],
        [0.3365, 0.1623],
        [0.6916, 0.9256],
        [0.1574, 0.4046],
        [0.7143, 0.3958],
        [0.3072, 0.6785],
        [0.3442, 0.8242],
        [0.4807, 0.5795],
        [0.7869, 0.2391],
        [0.8203, 0.9767],
        [0.6361, 0.5395],
        [0.9245, 0.5752],
        [0.0582, 0.3420],
        [0.4908, 0.6234],
        [0.8633, 0.9506],
        [0.9131, 0.1951],
        [0.0342, 0.4091],
        [0.6588, 0.2756],
        [0.1228, 0.9335]]  # 좌표값 리스트
rewards =  [40, 55, 60, 76, 42, 36,  9,  2, 51, 23, 33, 78, 27, 41, 84, 51,  6, 76, 38, 57, 71, 32, 98, 16, 99, 31, 46, 52, 56, 10,  2, 82, 37, 44, 23, 12,
        33, 86, 57, 58, 22, 85, 38, 34, 64, 42, 89, 50, 54, 77, 95, 12, 90, 11,52, 87, 47, 93, 71, 69,  3, 86, 21,  9, 86, 29, 65, 80, 11, 80, 61, 22,
        38, 93, 77, 91, 26,  7, 39,  2, 46, 16, 66, 54, 92,  4, 32, 84, 80, 73, 41, 56, 45, 84, 20, 52, 61, 30, 33,  3]   # 리워드 리스트
max_distance = 50

result = pctsp_with_rewards(coordinates, rewards, max_distance)
if isinstance(result, tuple):
    total_distance, total_reward = result
    print(f"Total Distance: {total_distance}")
    print(f"Total Reward: {total_reward}")
else:
    print(result)

No solution found


In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

def create_data_model():
    data = {}
    data['num_nodes'] = 20
    data['node_coordinates'] = [
        (0.8399, 0.3007), (0.4468, 0.0890), (0.4920, 0.6809), (0.6088, 0.2040),
        (0.6129, 0.1492), (0.4122, 0.7854), (0.5306, 0.3192), (0.1819, 0.5648),
        (0.7146, 0.9010), (0.0962, 0.0210), (0.6243, 0.8617), (0.1295, 0.2442),
        (0.9077, 0.2649), (0.5887, 0.7566), (0.6873, 0.3199), (0.4174, 0.9208),
        (0.4273, 0.9045), (0.3627, 0.8007), (0.2802, 0.5079), (0.1866, 0.2817)
    ]
    data['node_rewards'] = [
        40, 55, 60, 76, 42, 36, 9, 2, 51, 23,
        33, 78, 27, 41, 84, 51, 6, 76, 38, 57
    ]
    data['max_travel_distance'] = 10
    return data

def create_distance_callback(data):
    def distance_callback(from_index, to_index):
        from_node = data['node_coordinates'][from_index]
        to_node = data['node_coordinates'][to_index]
        return ((from_node[0] - to_node[0])**2 + (from_node[1] - to_node[1])**2)**0.5
    return distance_callback

def create_reward_callback(data):
    def reward_callback(from_index):
        return data['node_rewards'][from_index]
    return reward_callback

def main():
    data = create_data_model()

    # Routing 모델 설정
    manager = pywrapcp.RoutingIndexManager(data['num_nodes'], 1, 0)
    routing = pywrapcp.RoutingModel(manager)

    # 거리와 리워드 함수 설정
    distance_callback = create_distance_callback(data)
    routing.SetArcCostEvaluatorOfAllVehicles(routing.RegisterTransitCallback(distance_callback))

    reward_callback = create_reward_callback(data)
    routing.AddDimensionWithVehicleCapacity(
        routing.RegisterUnaryTransitCallback(reward_callback), 0, [data['max_travel_distance']] * data['num_nodes'], True, 'Reward'
    )

    # 최적화 수행
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    solution = routing.SolveWithParameters(search_parameters)

    # 결과 출력
    if solution:
        print('총 리워드 값:', routing.GetArcCostForAllVehicles())
        print('총 여행 거리:', routing.GetTotalDistance())
        print('총 실행 시간(ms):', routing.solver().wall_time())

if __name__ == '__main__':
    main()


: 